In [53]:
import os
import sys
from core.utils import load_dict_from_yaml
from core.services.backend_api_client import BackendAPIClient

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [54]:
top_configs = [
    load_dict_from_yaml(file_name='macd_bb_1.yml', folder=root_path+"/config"),
    load_dict_from_yaml(file_name='macd_bb_2.yml', folder=root_path+"/config"),
]

In [55]:
client = BackendAPIClient()
list_accounts = await client.get_accounts()
account_portfolios = await client.get_accounts_state()
total_quote_balance = 100

asym_babel_each_strat_balance = {
     "macd_bb_1": 0.8,
     "macd_bb_2": 0.2
}
# Setup config amount based on the strategy
for config in top_configs:
     config["total_amount_quote"] = total_quote_balance * asym_babel_each_strat_balance[config["id"]]
     await client.add_controller_config(config)

In [56]:
await client.deploy_script_with_controllers(
    bot_name="test_2_strategy",
    controller_configs=[config["id"] + ".yml" for config in top_configs],
    image_name="hummingbot/hummingbot:latest",
    max_global_drawdown=50,
    max_controller_drawdown=20,
)

{'success': True,
 'message': 'Instance hummingbot-test_2_strategy-2025.04.25_03.42 created successfully.'}